In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import os
# %cd '/content/drive/My Drive/MindEye'
# os.chdir('/content/drive/My Drive/MindEye')

In [ ]:
# !pip install git+https://github.com/openai/CLIP.git

In [ ]:
import torch
import torch.nn as nn
from transformers import CLIPModel, CLIPProcessor, GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import pandas as pd
import csv
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
import requests
from model import Voxel2Blip
from torchvision import transforms
from transformers import BlipProcessor, BlipForConditionalGeneration
from transformers import BlipImageProcessor
from diffusers.models import AutoencoderKL
from utils import load_image, save_image, encode_img, decode_img, to_PIL
from transformers.image_utils import OPENAI_CLIP_MEAN,OPENAI_CLIP_STD

In [ ]:
dataset_path = '2023-Machine-Learning-Dataset/'
training_path = dataset_path + 'subj0{}/training_split/'
training_fmri_path = training_path + 'training_fmri/'
training_images_path = training_path + 'training_images/'
testing_path = dataset_path + 'subj0{}/test_split/test_fmri/'
image_infos_path = dataset_path + 'image_infos/subj0{}_infos_train.csv'

In [ ]:
class MyDataset(Dataset):
  def __init__(self, fmri_data,images_folder, images_data, transform=None ,device="cpu"):
    self.fmri_data = torch.from_numpy(fmri_data)
    self.images_data = images_data
    self.images_folder = images_folder
    self.image_paths = []
    for i in range(5000):
      self.image_paths.append(f"{images_folder}/{i}.png")
    self.transform = transform

  def __len__(self):
    return len(self.fmri_data)

  def __getitem__(self, idx):
    fmri = self.fmri_data[idx]
    image = self.images_data[idx]
    image_path = self.image_paths[idx]
    image_real = load_image(image_path)

    if(self.transform):
      image_real = self.transform(image_real)

    return fmri, image, image_real
transform = transforms.Resize([512, 512])

# Load dataset, now only subj01
lh = np.load(training_path.format(1) + 'training_fmri/lh_training_fmri.npy')
rh = np.load(training_path.format(1) + 'training_fmri/rh_training_fmri.npy')
lrh = np.concatenate((lh, rh), axis=1)

images_bilp = torch.load('subj01_image_blip_encoded.pt')['images']

my_dataset = MyDataset(lrh,training_images_path.format(1), images_bilp, transform=transform)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
modelGeneration = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large", torch_dtype=torch.float16).to(device)
imgProcessor = BlipImageProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae").to(device)

In [ ]:
generator = torch.Generator().manual_seed(42)
trainset, validset = random_split(my_dataset, [0.9, 0.1], generator=generator)
# build dataloader
train_dataloader = DataLoader(trainset, batch_size=16, shuffle=False, num_workers=0)
val_dataloader = DataLoader(validset, batch_size=16, shuffle=False, num_workers=0)

In [ ]:
model = Voxel2Blip().to(device)
checkpoint = torch.load('./ModelsBlip/550.3_f1')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()
del checkpoint

In [ ]:
for train_i, data in enumerate(val_dataloader):
    voxels, images, originImages = data
    voxels = voxels.to(device).float()
    images = images.to(device).float()
    originImages = originImages.to(device).float()
    out_encoded = model(voxels)
    break

In [ ]:
for image, encoded, originImage in zip(images,out_encoded,originImages):
    imgOrigin = decode_img(image.unsqueeze(0), vae)
    img = decode_img(encoded.unsqueeze(0), vae)
    # print(imgProcessor(img,do_normalize = False)['pixel_values'][0])
    # break
    
    # encodedNormalized = imgProcessor(img,do_normalize = False)['pixel_values'][0]
    # encodedNormalized = imgProcessor.normalize(decode_img(encoded.unsqueeze(0), vae).cpu().numpy(),mean = OPENAI_CLIP_MEAN,std = OPENAI_CLIP_STD)
    # out = modelGeneration.generate(torch.from_numpy(encodedNormalized).to(device))
    out = modelGeneration.generate(img).to(device)
    print(processor.decode(out[0], skip_special_tokens=True))
    imgOrigin = ((np.transpose(imgOrigin[0].cpu().numpy(), (1, 2, 0))) * 255).astype(np.uint8)
    plt.imshow(imgOrigin)
    plt.show()
    

In [ ]:


img = load_image("./outputImg/origin_1.png")
print(img.shape)
# encoded = encode_img(img.to(device), vae.to(device))
img = (img*255).int()
inputs = processor(img.squeeze(0), return_tensors="pt").to(device, torch.float16)

out = modelGeneration.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))

In [ ]:
image = imgProcessor(img,do_normalize = False)['pixel_values'][0]

In [ ]:

encoded.shape

In [ ]:
decode_img(encoded, vae)[0].shape

In [ ]:
out = model.generate(decode_img(encoded, vae))
print(processor.decode(out[0], skip_special_tokens=True))